<a href="https://colab.research.google.com/github/KaveGhsdn/Data_Science/blob/main/Cohort_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import auth
import pandas as pd
auth.authenticate_user()

In [4]:
query = """
SELECT *
FROM physionet-data.mimiciv_hosp.patients
"""
patients_df = pd.read_gbq(query, project_id='physionet-data-403216', dialect='standard')
patients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299712 entries, 0 to 299711
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   subject_id         299712 non-null  Int64 
 1   gender             299712 non-null  object
 2   anchor_age         299712 non-null  Int64 
 3   anchor_year        299712 non-null  Int64 
 4   anchor_year_group  299712 non-null  object
 5   dod                29076 non-null   dbdate
dtypes: Int64(3), dbdate(1), object(2)
memory usage: 14.6+ MB


In [5]:
query = """
SELECT *
FROM physionet-data.mimiciv_hosp.admissions
"""
admissions_df = pd.read_gbq(query, project_id='physionet-data-403216', dialect='standard')
admissions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431231 entries, 0 to 431230
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   subject_id            431231 non-null  Int64         
 1   hadm_id               431231 non-null  Int64         
 2   admittime             431231 non-null  datetime64[us]
 3   dischtime             431231 non-null  datetime64[us]
 4   deathtime             8598 non-null    datetime64[us]
 5   admission_type        431231 non-null  object        
 6   admit_provider_id     431227 non-null  object        
 7   admission_location    431231 non-null  object        
 8   discharge_location    312076 non-null  object        
 9   insurance             431231 non-null  object        
 10  language              431231 non-null  object        
 11  marital_status        421998 non-null  object        
 12  race                  431231 non-null  object        
 13 

In [6]:
query = """
SELECT *
FROM physionet-data.mimiciv_hosp.transfers
"""
transfers_df = pd.read_gbq(query, project_id='physionet-data-403216', dialect='standard')
transfers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1890972 entries, 0 to 1890971
Data columns (total 7 columns):
 #   Column       Dtype         
---  ------       -----         
 0   subject_id   Int64         
 1   hadm_id      Int64         
 2   transfer_id  Int64         
 3   eventtype    object        
 4   careunit     object        
 5   intime       datetime64[us]
 6   outtime      datetime64[us]
dtypes: Int64(3), datetime64[us](2), object(2)
memory usage: 106.4+ MB


In [7]:
query = """
SELECT *
FROM physionet-data.mimiciv_icu.icustays
"""
icustays_df = pd.read_gbq(query, project_id='physionet-data-403216', dialect='standard')
icustays_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73181 entries, 0 to 73180
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   subject_id      73181 non-null  Int64         
 1   hadm_id         73181 non-null  Int64         
 2   stay_id         73181 non-null  Int64         
 3   first_careunit  73181 non-null  object        
 4   last_careunit   73181 non-null  object        
 5   intime          73181 non-null  datetime64[us]
 6   outtime         73181 non-null  datetime64[us]
 7   los             73181 non-null  float64       
dtypes: Int64(3), datetime64[us](2), float64(1), object(2)
memory usage: 4.7+ MB


In [8]:
# Rename 'inittime' and 'outtime' columns in 'transfers_df'
transfers_df = transfers_df.rename(columns={'intime': 'intime_transfers', 'outtime': 'outtime_transfers'})
icustays_df = icustays_df.rename(columns={'intime': 'intime_icu', 'outtime': 'outtime_icu'})

In [9]:
# Merge icustays_df with transfers_df
icu_transfer = pd.merge(icustays_df, transfers_df, on=['subject_id', 'hadm_id'], how='inner')
icu_transfer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400610 entries, 0 to 400609
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   subject_id         400610 non-null  Int64         
 1   hadm_id            400610 non-null  Int64         
 2   stay_id            400610 non-null  Int64         
 3   first_careunit     400610 non-null  object        
 4   last_careunit      400610 non-null  object        
 5   intime_icu         400610 non-null  datetime64[us]
 6   outtime_icu        400610 non-null  datetime64[us]
 7   los                400610 non-null  float64       
 8   transfer_id        400610 non-null  Int64         
 9   eventtype          400610 non-null  object        
 10  careunit           327429 non-null  object        
 11  intime_transfers   400610 non-null  datetime64[us]
 12  outtime_transfers  327429 non-null  datetime64[us]
dtypes: Int64(4), datetime64[us](4), float64(1), 

In [10]:
i = icustays_df['subject_id'].nunique()
t = transfers_df['subject_id'].nunique()
a = admissions_df['subject_id'].nunique()
p = patients_df['subject_id'].nunique()
print(f"icu: {i}\ntransfer: {t}\nadmission: {a}\npatients: {p}")

icu: 50920
transfer: 299712
admission: 180733
patients: 299712


In [11]:
# Merge icu_transfer with admissions_df
icu_transfer_admission = pd.merge(icu_transfer, admissions_df, on=['subject_id', 'hadm_id'], how='inner')
icu_transfer_admission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400610 entries, 0 to 400609
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   subject_id            400610 non-null  Int64         
 1   hadm_id               400610 non-null  Int64         
 2   stay_id               400610 non-null  Int64         
 3   first_careunit        400610 non-null  object        
 4   last_careunit         400610 non-null  object        
 5   intime_icu            400610 non-null  datetime64[us]
 6   outtime_icu           400610 non-null  datetime64[us]
 7   los                   400610 non-null  float64       
 8   transfer_id           400610 non-null  Int64         
 9   eventtype             400610 non-null  object        
 10  careunit              327429 non-null  object        
 11  intime_transfers      400610 non-null  datetime64[us]
 12  outtime_transfers     327429 non-null  datetime64[us]
 13 

In [12]:
# Merge icu_transfer_admission with patients_df
icu_transfer_admission_patient = pd.merge(icu_transfer_admission, patients_df, on=['subject_id'], how='inner')
icu_transfer_admission_patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400610 entries, 0 to 400609
Data columns (total 32 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   subject_id            400610 non-null  Int64         
 1   hadm_id               400610 non-null  Int64         
 2   stay_id               400610 non-null  Int64         
 3   first_careunit        400610 non-null  object        
 4   last_careunit         400610 non-null  object        
 5   intime_icu            400610 non-null  datetime64[us]
 6   outtime_icu           400610 non-null  datetime64[us]
 7   los                   400610 non-null  float64       
 8   transfer_id           400610 non-null  Int64         
 9   eventtype             400610 non-null  object        
 10  careunit              327429 non-null  object        
 11  intime_transfers      400610 non-null  datetime64[us]
 12  outtime_transfers     327429 non-null  datetime64[us]
 13 

In [13]:
print(icu_transfer_admission_patient.columns)

Index(['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit',
       'intime_icu', 'outtime_icu', 'los', 'transfer_id', 'eventtype',
       'careunit', 'intime_transfers', 'outtime_transfers', 'admittime',
       'dischtime', 'deathtime', 'admission_type', 'admit_provider_id',
       'admission_location', 'discharge_location', 'insurance', 'language',
       'marital_status', 'race', 'edregtime', 'edouttime',
       'hospital_expire_flag', 'gender', 'anchor_age', 'anchor_year',
       'anchor_year_group', 'dod'],
      dtype='object')


In [14]:
general_df = icu_transfer_admission_patient.copy()

In [15]:
general_df['subject_id'].nunique()

50920

In [16]:
expire_flag_1_df = general_df[general_df['hospital_expire_flag'] == 1]
unique_subject_ids_count = expire_flag_1_df['subject_id'].nunique()
print("Number of unique subject_ids with hospital_expire_flag equal to 1:", unique_subject_ids_count)

Number of unique subject_ids with hospital_expire_flag equal to 1: 6966


In [17]:
los_less_5 = general_df[general_df['los'] < 0.5]
unique_subject_ids_count = los_less_5['subject_id'].nunique()
print("Number of unique subject_ids with los less than 0.5 Day:", unique_subject_ids_count)

Number of unique subject_ids with los less than 0.5 Day: 2937


In [18]:
above_91 = general_df[general_df['anchor_age'] == 91]
unique_subject_ids_count = above_91['subject_id'].nunique()
print("Number of unique subject_ids with age above from 89:", unique_subject_ids_count)

Number of unique subject_ids with age above from 89: 2158


In [19]:
filtered_df = general_df[(general_df['hospital_expire_flag'] != 1) &
                         (general_df['los'] >= 0.5) &
                         (general_df['anchor_age'] != 91)]


In [20]:
filtered_df['subject_id'].nunique()

42985

In [21]:
filtered_df.columnss

Index(['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit',
       'intime_icu', 'outtime_icu', 'los', 'transfer_id', 'eventtype',
       'careunit', 'intime_transfers', 'outtime_transfers', 'admittime',
       'dischtime', 'deathtime', 'admission_type', 'admit_provider_id',
       'admission_location', 'discharge_location', 'insurance', 'language',
       'marital_status', 'race', 'edregtime', 'edouttime',
       'hospital_expire_flag', 'gender', 'anchor_age', 'anchor_year',
       'anchor_year_group', 'dod'],
      dtype='object')

In [22]:
missing_values = filtered_df.isnull().sum()
print(missing_values)


subject_id                   0
hadm_id                      0
stay_id                      0
first_careunit               0
last_careunit                0
intime_icu                   0
outtime_icu                  0
los                          0
transfer_id                  0
eventtype                    0
careunit                 60462
intime_transfers             0
outtime_transfers        60462
admittime                    0
dischtime                    0
deathtime               332447
admission_type               0
admit_provider_id           12
admission_location           0
discharge_location        1616
insurance                    0
language                     0
marital_status           18709
race                         0
edregtime               111272
edouttime               111272
hospital_expire_flag         0
gender                       0
anchor_age                   0
anchor_year                  0
anchor_year_group            0
dod                     227313
dtype: i

In [23]:
unique_careunits = filtered_df['careunit'].unique()
for careunit in unique_careunits:
    print(careunit)


Neuro Stepdown
Discharge Lounge
None
Neurology
Trauma SICU (TSICU)
PACU
Emergency Department
Neuro Surgical Intensive Care Unit (Neuro SICU)
Neuro Intermediate
Hematology/Oncology
Medicine/Cardiology
Coronary Care Unit (CCU)
Surgical Intensive Care Unit (SICU)
Med/Surg
Medicine
Medical Intensive Care Unit (MICU)
Cardiac Surgery
Medical/Surgical Intensive Care Unit (MICU/SICU)
Med/Surg/GYN
Cardiac Vascular Intensive Care Unit (CVICU)
Vascular
Hematology/Oncology Intermediate
Surgery
Med/Surg/Trauma
Transplant
Emergency Department Observation
Cardiology
Surgery/Trauma
Medicine/Cardiology Intermediate
Surgery/Pancreatic/Biliary/Bariatric
Cardiology Surgery Intermediate
Observation
Medical/Surgical (Gynecology)
Obstetrics Postpartum
Thoracic Surgery
Obstetrics (Postpartum & Antepartum)
Labor & Delivery
Obstetrics Antepartum
Unknown


In [24]:
value_counts = filtered_df['careunit'].value_counts()
for careunit, count in value_counts.items():
    print(f"{careunit}: {count}")


Emergency Department: 38787
Medicine: 27640
Medical Intensive Care Unit (MICU): 18852
Cardiac Vascular Intensive Care Unit (CVICU): 16754
Medical/Surgical Intensive Care Unit (MICU/SICU): 14086
Surgical Intensive Care Unit (SICU): 13945
Neurology: 13660
Cardiac Surgery: 12860
Medicine/Cardiology: 11640
Discharge Lounge: 11281
Trauma SICU (TSICU): 11233
PACU: 10371
Coronary Care Unit (CCU): 9651
Med/Surg: 9440
Transplant: 8043
Hematology/Oncology: 5383
Med/Surg/Trauma: 5029
Vascular: 4686
Neuro Intermediate: 3877
Cardiology Surgery Intermediate: 3672
Surgery/Trauma: 3204
Med/Surg/GYN: 3151
Emergency Department Observation: 2580
Medicine/Cardiology Intermediate: 2278
Neuro Surgical Intensive Care Unit (Neuro SICU): 2188
Neuro Stepdown: 1932
Hematology/Oncology Intermediate: 1451
Surgery/Pancreatic/Biliary/Bariatric: 1397
Medical/Surgical (Gynecology): 825
Surgery: 712
Thoracic Surgery: 359
Labor & Delivery: 312
Obstetrics (Postpartum & Antepartum): 278
Observation: 152
Cardiology: 111
Ob

In [25]:
sorted_filtered_df = filtered_df.sort_values(by=['subject_id', 'hadm_id', 'intime_transfers', 'outtime_transfers'])

In [26]:
sorted_filtered_df.head(5)

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime_icu,outtime_icu,los,transfer_id,eventtype,...,marital_status,race,edregtime,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod
256323,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,39866888,ED,...,MARRIED,WHITE,2157-11-18 17:38:00,2157-11-19 01:24:00,0,F,55,2157,2011 - 2013,NaT
256326,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,37058438,admit,...,MARRIED,WHITE,2157-11-18 17:38:00,2157-11-19 01:24:00,0,F,55,2157,2011 - 2013,NaT
256322,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,37067082,transfer,...,MARRIED,WHITE,2157-11-18 17:38:00,2157-11-19 01:24:00,0,F,55,2157,2011 - 2013,NaT
256324,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,35343802,transfer,...,MARRIED,WHITE,2157-11-18 17:38:00,2157-11-19 01:24:00,0,F,55,2157,2011 - 2013,NaT
256321,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,30437372,transfer,...,MARRIED,WHITE,2157-11-18 17:38:00,2157-11-19 01:24:00,0,F,55,2157,2011 - 2013,NaT


In [27]:
from google.colab import auth
from google.colab import drive
auth.authenticate_user()
drive.mount('/content/gdrive')
sorted_filtered_df.to_excel('/content/gdrive/My Drive/sorted_filtered_df.xlsx', index=False)

Mounted at /content/gdrive
